<a href="https://colab.research.google.com/github/SaketDawgotra/Aggregation_python/blob/main/NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
SOS_token = 0
EOS_token = 1


class Language:
    def __init__(self, name):
        self.name = name
        self.word_index_mapping = {}
        self.word_count_mapping = {}
        self.index_word_mapping = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def add_sentence(self, sentence):
        for word in sentence.split(' '):
            self.add_word(word)

    def add_word(self, word):
        if word not in self.word_index_mapping:
            self.word_index_mapping[word] = self.n_words
            self.word_count_mapping[word] = 1
            self.index_word_mapping[self.n_words] = word
            self.n_words += 1
        else:
            self.word_count_mapping[word] += 1

In [ ]:
MAX_LENGTH = 50
eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)
class Utils:
  def unicodeToAscii(self, s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

  def format(self, s):
    s = self.unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s
  
  def filter_pair(self, p):
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

  def filter_pairs(self, pairs):
    return [pair for pair in pairs if self.filter_pair(pair)]

utils = Utils()

In [ ]:
def read_languages(lang1, lang2):
    print("Reading lines for language ", lang1, "to language ", lang2 )
    lines = open('sample_data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[utils.format(s) for s in l.split('\t')] for l in lines]
    print(pairs[0])

    input_lang = Language(lang1)
    output_lang = Language(lang2)

    return input_lang, output_lang, pairs

In [ ]:
def prepare_data(lang1, lang2):
    input_lang, output_lang, pairs = read_languages(lang1, lang2)
    print("Read %s sentence pairs" % len(pairs))
    pairs = utils.filter_pairs(pairs)
    # print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
      input_lang.add_sentence(pair[0])
      output_lang.add_sentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepare_data('eng', 'fra')
print(random.choice(pairs))

Reading lines for language  eng to language  fra
['go .', 'va !']
Read 135842 sentence pairs
Counting words...
Counted words:
eng 13038
fra 21325
['i left a message for tom .', 'j ai laisse un message pour tom .']


In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [ ]:
def get_indexes_from_sentences(lang, sentence):
    return [lang.word_index_mapping[word] for word in sentence.split(' ')]


def get_tensor_from_sentence(lang, sentence):
    indexes = get_indexes_from_sentences(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def get_tensors_from_pair(pair):
    input_tensor = get_tensor_from_sentence(input_lang, pair[0])
    target_tensor = get_tensor_from_sentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [ ]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [ ]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
def trainIters(encoder, decoder, n_iters=150000, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [get_tensors_from_pair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    # showPlot(plot_losses)

In [ ]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = get_tensor_from_sentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index_word_mapping[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [ ]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)



In [ ]:
trainIters(encoder1, attn_decoder1)

0m 25s (- 63m 34s) (1000 0%) 5.1466
0m 41s (- 50m 36s) (2000 1%) 4.8097
0m 56s (- 46m 15s) (3000 2%) 4.7266
1m 12s (- 43m 55s) (4000 2%) 4.5294
1m 28s (- 42m 38s) (5000 3%) 4.5035
1m 44s (- 41m 46s) (6000 4%) 4.4070
2m 0s (- 41m 7s) (7000 4%) 4.4198
2m 17s (- 40m 41s) (8000 5%) 4.3603
2m 33s (- 40m 10s) (9000 6%) 4.2831
2m 50s (- 39m 44s) (10000 6%) 4.2521
3m 7s (- 39m 23s) (11000 7%) 4.2685
3m 23s (- 39m 3s) (12000 8%) 4.1972
3m 40s (- 38m 39s) (13000 8%) 4.0876
3m 56s (- 38m 21s) (14000 9%) 4.0556
4m 13s (- 37m 59s) (15000 10%) 3.9961
4m 30s (- 37m 43s) (16000 10%) 3.9831
4m 46s (- 37m 24s) (17000 11%) 3.9241
5m 3s (- 37m 6s) (18000 12%) 3.8829
5m 20s (- 36m 49s) (19000 12%) 3.8326
5m 36s (- 36m 29s) (20000 13%) 3.7858
5m 53s (- 36m 11s) (21000 14%) 3.7928
6m 10s (- 35m 53s) (22000 14%) 3.7722
6m 26s (- 35m 35s) (23000 15%) 3.7124
6m 43s (- 35m 15s) (24000 16%) 3.7140
6m 59s (- 34m 58s) (25000 16%) 3.6896
7m 16s (- 34m 42s) (26000 17%) 3.7455
7m 33s (- 34m 27s) (27000 18%) 3.6573
7m 

In [ ]:
evaluateRandomly(encoder1, attn_decoder1)

> i thought you of all people would understand my decision .
= je pensais que toi entre tous comprendrait ma decision .
< je pensais que tu me pris tous mes gens . . <EOS>

> i dislike coffee .
= je n aime pas le cafe .
< je m du cafe . <EOS>

> do you know who wrote this book ?
= est ce que tu sais qui a ecrit ce livre ?
< savez vous qui ecrit ce qui ecrit ecrit ? <EOS>

> she made it plain that she wanted to go to college .
= elle fit clairement comprendre qu elle voulait aller a l universite .
< elle a qu elle voulait aller a l ecole . <EOS>

> tom is even scared of mary .
= tom a meme peur de mary .
< tom est mort de l . <EOS>

> you look depressed . did something happen ?
= tu as l air deprimee quelque chose est il survenu ?
< tu as l air ce qui il a quelque chose ? <EOS>

> he kicked the ball with his foot .
= il tira dans le ballon avec son pied .
< il a son le le le le . . . <EOS>

> it is very kind of you to invite me .
= c est tres aimable a vous de m inviter .
< il est tres 